<a href="https://www.kaggle.com/code/azizbekboboqulov/airfare-price-prediction?scriptVersionId=135980664" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

## Dataset

In [3]:
train=pd.read_csv("/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv")
test=pd.read_csv("/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv")
submission=pd.read_csv("/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv")
train.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
1,2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
2,3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
3,4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
4,5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


In [4]:
train['stops'] = train['stops'].replace({'one': 1, 'zero': 0, 'two_or_more': 2})
train['class'] = train['class'].replace({'Economy': 0, 'Business': 1})
train.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,1,Vistara,UK-810,Bangalore,Early_Morning,1,Night,Mumbai,0,14.25,21,7212
1,2,SpiceJet,SG-5094,Hyderabad,Evening,0,Night,Kolkata,0,1.75,7,5292
2,3,Vistara,UK-846,Bangalore,Morning,1,Evening,Delhi,1,9.58,5,60553
3,4,Vistara,UK-706,Kolkata,Morning,1,Evening,Hyderabad,0,6.75,28,5760
4,5,Indigo,6E-5394,Chennai,Early_Morning,0,Morning,Mumbai,0,2.00,4,10712


In [5]:
test['stops'] = test['stops'].replace({'one': 1, 'zero': 0, 'two_or_more': 2})
test['class'] = test['class'].replace({'Economy': 0, 'Business': 1})
test.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
0,1,Air_India,AI-765,Kolkata,Evening,1,Night,Delhi,1,28.25,2
1,2,Vistara,UK-747,Delhi,Early_Morning,1,Night,Mumbai,1,13.83,34
2,3,Air_India,AI-570,Mumbai,Early_Morning,0,Early_Morning,Chennai,1,2.00,30
3,4,AirAsia,I5-974,Hyderabad,Night,1,Late_Night,Delhi,0,5.17,26
4,5,Air_India,AI-770,Kolkata,Night,1,Afternoon,Mumbai,0,16.33,35


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                20000 non-null  int64  
 1   airline           20000 non-null  object 
 2   flight            20000 non-null  object 
 3   source_city       20000 non-null  object 
 4   departure_time    20000 non-null  object 
 5   stops             20000 non-null  int64  
 6   arrival_time      20000 non-null  object 
 7   destination_city  20000 non-null  object 
 8   class             20000 non-null  int64  
 9   duration          20000 non-null  float64
 10  days_left         20000 non-null  int64  
 11  price             20000 non-null  int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 1.8+ MB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                5000 non-null   int64  
 1   airline           5000 non-null   object 
 2   flight            5000 non-null   object 
 3   source_city       5000 non-null   object 
 4   departure_time    5000 non-null   object 
 5   stops             5000 non-null   int64  
 6   arrival_time      5000 non-null   object 
 7   destination_city  5000 non-null   object 
 8   class             5000 non-null   int64  
 9   duration          5000 non-null   float64
 10  days_left         5000 non-null   int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 429.8+ KB


## Preparation for ML

In [8]:
X_train=train.drop('price', axis=1)
y=train['price']

train_set, test_set=train_test_split(train, test_size=0.2, random_state=42)

In [9]:
num_attribs=['stops', 'class', 'duration', 'days_left']
cat_attribs=['airline', 'flight', 'source_city', 'departure_time', 'arrival_time', 'destination_city']

num_pipeline = Pipeline([
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', cat_pipeline, cat_attribs)
])

In [10]:
X_prepared = full_pipeline.fit_transform(X_train)
X_prepared.toarray()[0:5,:]

array([[ 0.19281382, -0.67396452,  0.28952803, ...,  0.        ,
         0.        ,  1.        ],
       [-2.30154354, -0.67396452, -1.45682723, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.19281382,  1.48375764, -0.36291029, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.19281382, -0.67396452, -0.75828512, ...,  1.        ,
         0.        ,  0.        ],
       [-2.30154354, -0.67396452, -1.42190012, ...,  0.        ,
         0.        ,  1.        ]])

## ML
### Decision Tree Regression

In [11]:
Tree_model = DecisionTreeRegressor()
Tree_model.fit(X_prepared, y)

DecisionTreeRegressor()

In [12]:
predict = Tree_model.predict(X_prepared)

lin_mse=mean_squared_error(y, predict)
lin_rmse=np.sqrt(lin_mse)
print(lin_rmse)

42.13976684795491


In [13]:
X_test=test_set.drop('price', axis=1)
y_test=test_set['price'].copy()

In [14]:
X_test_prepared = full_pipeline.transform(X_test)

predict = Tree_model.predict(X_test_prepared)

lin_mse=mean_squared_error(y_test, predict)
lin_rmse=np.sqrt(lin_mse)
print(lin_rmse)

13.858681845687922


### Random Forest Regression

In [15]:
RF_model = RandomForestRegressor()
RF_model.fit(X_prepared, y)

RandomForestRegressor()

In [16]:
y_predicted = RF_model.predict(X_test_prepared)

lin_mse=mean_squared_error(y_test, y_predicted)
lin_rmse=np.sqrt(lin_mse)
print(lin_rmse)

1193.0586196156587


In [17]:
test_prepared = full_pipeline.transform(test)
test_predict = RF_model.predict(test_prepared)

In [18]:
submission['price']=test_predict
submission

,id,price
0,1,52763.37
1,2,61648.36
2,3,23342.64
3,4,2433.15
4,5,5689.53
...,...,...
4995,4996,60978.00
4996,4997,4453.69
4997,4998,50278.86
4998,4999,50505.75


In [19]:
submission.to_csv('submission_csv', index=False)